In [2]:
from PIL import Image, ImageFilter
from google.cloud import vision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import glob
import base64
import io
import os

#Funciones generales para uso global
import numpy as np
import time
from shapely.geometry import Polygon, Point
import geopandas as gpd
from shapely.ops import cascaded_union
import rasterio
from rasterio.mask import mask
from shapely import geometry
import pandas as pd
from geotiff import GeoTiff


#Funciones empleadas en la detección de texto
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

import rasterio
from rasterio.enums import Resampling

# Model v1: Split the image, detect text areas, and convert to text strings

## Splitting the image

In [2]:
# Takes a Rasterio dataset and splits it into squares of dimensions squareDim * squareDim
def splitImageIntoCells(img, filename, num_imgs=3):
    squareDim_wide = img.shape[1] // num_imgs
    squareDim_height = img.shape[0] // num_imgs
    
    numberOfCellsWide = img.shape[1] // squareDim_wide
    numberOfCellsHigh = img.shape[0] // squareDim_height
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim_height
        for wc in range(numberOfCellsWide):
            x = wc * squareDim_wide
            geom = getTileGeom(img.transform, x, y, squareDim_wide, squareDim_height)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1

# Generate a bounding box from the pixel-wise coordinates using the original datasets transform property
def getTileGeom(transform, x, y, squareDim_wide, squareDim_height):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim_wide, y + squareDim_height) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])

# Crop the dataset using the generated box and write it out as a GeoTIFF
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+"_"+str(count))

# Write the passed in dataset as a GeoTIFF
def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": 'EPSG:4686'})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [3]:
def detect_text(bytes_img):
    """
    Uses the Google Vision API to extract text from
    an image.
    
    Arguments
    ---------
    file_path: str
               path of the image to process.
    
    Outputs
    -------
    response: AnnotateImageResponse object
              json like format with bounding box and other
              relevant information.
    text: str
          text extracted from the image.
    """
    client = vision.ImageAnnotatorClient()    
    image = vision.Image(content=bytes_img)
    response = client.document_text_detection(image=image)
    text = response.full_text_annotation.text
    
    return response, text


def getbytesimg_from_path(filepath):
    """
    Obtains the bytes base64 format of an image from a 
    local file path.
    
    Arguments
    ---------
    filepath: str
              Path of the image file to convert
    
    Output
    ------
    bytes_img: bytes
               base64 format of the image
    """
    with open(filepath, "rb") as image_file:
        bytes_img = image_file.read()
    
    return bytes_img

In [4]:
def split_images(filepath, num_imgs=3):
    """
    Split a large image into a grid of 3x3
    smaller images.
    
    Arguments:
    ---------
    filepath: str
              file path of the large image
    num_imgs: int (optional)
              Number of rows and columns of the grid
              
    Output
    ------
    None
    
    """
    img = rasterio.open(filepath)
    splitImageIntoCells(img, "split_out/output_data")

In [5]:
def geotif_to_jpeg(tif_filename):
    """
    Converts geotif image from disk and saves it 
    in the same folder in jpeg format.
    
    Arguments
    ---------
    tif_filename: str
                  path of the tif file to convert
    
    Output
    ------
    None
    
    """
    with rasterio.open(tif_filename) as infile:    
        profile = infile.profile    
        profile['driver']='JPEG'
        jpeg_filename = tif_filename.replace(".tif", ".jpeg")
        with rasterio.open(jpeg_filename, 'w', **profile) as dst:
            dst.write(infile.read())

In [6]:
def get_text_coords(response):
    """
    Extracts the geographic names and bounding boxes coordinates
    from an AnnotateImageResponse object (Google).
    
    Arguments
    ---------
    response: AnnotateImageResponse (Google Vision API)
              Response object after calling the document_text_detection
              function
              
    Outputs
    -------
    palabras_google: list
                     List of strings containing the geographic names
    boundings_google: list
                      List of lists of 4 vertices that define the bounding 
                      box of each geographic name
    confidence_google: list
                       List of floats representing the confidence of the 
                       Google API at detecting each handwritten text
    """
    
    palabras_google = []
    boundings_google = []
    confidence_google = []
    
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            palabra_google = ''
            boundings_google.append(block.bounding_box)
            confidence_google.append(block.confidence)
            for parrafos in block.paragraphs:
                for palabras in parrafos.words:
                    for simbolo in palabras.symbols:
                        palabra_google = palabra_google+simbolo.text
                    palabra_google = palabra_google+' '
            palabras_google.append(palabra_google.rstrip())
            
    return palabras_google, boundings_google, confidence_google

In [7]:
def get_coords(boundings, word_index, point_index, axis):
    """
    Returns x or y coordinate from the list of vertices.
    
    Arguments
    ---------
    boundings: list
               List of bounding boxes with the vertices of 
               each bounding box
    word_index: int
                Index of the desired word starting at 0
    point_index: int
                 Index of the desired point (from 0 to 3)
    coord: str
           Either x or y
           
    Output
    ------
    coordinate: float
                Coordinate of the specified word, point, and axis
    
    """
    if axis == "x":
        coordinate = boundings[word_index].vertices[point_index].x
    else:
        coordinate = boundings[word_index].vertices[point_index].y
    
    return coordinate

In [8]:
def create_geometries(filepath):
    """
    Splits the image in a grid, uses Google Vision API to extract text
    and the bounding boxes. This method creates geojson files with 
    the geographic names, their respective geodesic coordinates, the 
    confidence, and the geometries of the bounding boxes.
    
    The geojson files are saved in the folder geometries.
    
    Arguments
    ---------
    filepath: str
              Path to the geotiff image file to process
              
    Output
    ------
    None
    """

    split_images(filepath)
    images_to_proces = os.listdir(path='split_out')

    for path in images_to_proces:
        if ('.tif' in path):
            geotif_to_jpeg("split_out/" + path)
            sub_image_jpeg = path.replace('.tif','.jpeg')
            response, text = detect_text(getbytesimg_from_path("split_out/" + sub_image_jpeg))
            words, boundings, confidence = get_text_coords(response)
            img = rasterio.open("split_out/" + sub_image_jpeg)

            # Generating the sub_geometries
            geometries = []
            centroids = []

            for i in range(len(words)):
                aux_polygon = Polygon([img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x")),
                                       img.xy(get_coords(boundings,i,1,"y"),get_coords(boundings,i,1,"x")),
                                       img.xy(get_coords(boundings,i,2,"y"),get_coords(boundings,i,2,"x")),
                                       img.xy(get_coords(boundings,i,3,"y"),get_coords(boundings,i,3,"x")),
                                       img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x"))])
                geometries.append(aux_polygon)
                centroids.append(aux_polygon.representative_point())

            sub_img_gdf = gpd.GeoDataFrame(columns=["toponimo_ocr","confidence",
                "centroide_longitud","centroide_latitud","geometry"], crs=str(img.crs))
            sub_img_gdf["geometry"] = geometries
            sub_img_gdf["toponimo_ocr"] = words
            sub_img_gdf["confidence"] = confidence
            sub_img_gdf["centroide_longitud"] = [x.coords[0][0] for x in centroids]
            sub_img_gdf["centroide_latitud"] = [x.coords[0][1] for x in centroids]
            name_geometry = 'geometries/'+sub_image_jpeg.replace('.jpeg','.geojson')
            try:
                sub_img_gdf.to_file(name_geometry, driver="GeoJSON")
            except:
                print('Empty text detected, no geometries generated!')

In [9]:
def combine_geometries(filepath):
    """
    Takes all the geojson files in the geometries folder and combine
    them into a single geojson file. It combines bounding boxes that 
    intersect each other and their respective geographic names. It 
    deletes the rows whose name has only numbers.
    
    Arguments
    ---------
    filepath: str
              Path of the original image
    
    Output
    ------
    all_toponyms_gdf: GeoDataFrame
                      GeoDataFrame with all the toponyms from the original
                      image. It contains the geographic name, geodesic 
                      coordinates of the centroid, confidence and the 
                      geometry of the bounding boxes.
    
    """
    org_img = rasterio.open(filepath)
    geometries_to_process = os.listdir(path='geometries')    
    rectangles = []
    all_toponyms_gdf = gpd.GeoDataFrame(columns=["toponimo_ocr","confidence",
                "centroide_longitud","centroide_latitud","geometry"],crs=str(org_img.crs))

    for geometry in geometries_to_process:
        if ".geojson" in geometry:
            file = gpd.read_file('geometries/' + geometry)
            all_toponyms_gdf = all_toponyms_gdf.append(file)

    all_toponyms_gdf.reset_index(drop=True, inplace=True)
    all_toponyms_gdf.to_file('text_detected.geojson',driver='GeoJSON')
    
    return all_toponyms_gdf

In [10]:
def get_image_corners(filepath):
    """
    Returns the geodesic coordinates of the original image.
    
    Arguments
    ---------
    filepath: str
              Path of the original image
    
    Output
    ------
    image_corners: dict
                   Dictionary with two keys: "upper_left" and "lower_left"
                   The values are tuples with the latitude and longitude
                   of the image corners.
    """
    original_img = rasterio.open(filepath)
    image_corners = {
        "upper_left": original_img.xy(0,0),
        "lower_right": original_img.xy(original_img.shape[0], original_img.shape[1])
    }
    
    return image_corners

In [11]:
def empty_folders():
    """
    Remove all temporary output files in the "split_out" and
    "geometries" folders. It also deletes the final output file
    detected_text.geojson.
    
    Arguments
    ---------
    None
    
    Output
    ------
    None
    """
    folders = ["split_out", "geometries"]
    
    for folder in folders:
        files = glob.glob(f"{folder}/*")
        for f in files:
            os.remove(f)
    try:
        os.remove("text_detected.geojson")
    except:
        print("text_detected.geojson already deleted!")

In [52]:
def union_geometries(geojson_toponimos):
    """
    It allows to detect the text divided organically by the subdivision 
    of the images and to unite the geometries and the detected text
    
    Arguments
    ---------
    geojson_toponimos: geopandas
                       Initial geopandas union of all split image
    
    Output: 
    ------
    new_geojseon: geopandas
                  geopandas with all union geometries and detected text
    
    """

    geojson_toponimos.reset_index(drop=True, inplace=True)
    new_geojseon = gpd.GeoDataFrame(columns=["toponimo_ocr","confidence","centroide_longitud","centroide_latitud",
                                             "geometry"],crs=geojson_toponimos.crs)
    while( len(geojson_toponimos)!=0 ):
        indexs = []
        new_elments =[]

        for i in range(0,len(geojson_toponimos)):
            if ( geojson_toponimos.geometry[0].intersects( geojson_toponimos.geometry[i] ) ):
                indexs.append(i)

        if ( len(indexs)==1):
            indexs = [0]
            new_geojseon = new_geojseon.append(geojson_toponimos.iloc[0:1])

        else:

            for j in range(0,4):
                for inde in range( len(indexs) ):
                    for i in range(0,len(geojson_toponimos)):
                        if ( geojson_toponimos.geometry[ indexs[inde] ].intersects( geojson_toponimos.geometry[i] ) ):
                            indexs.append(i)
                indexs = list(set(indexs))

            geom_list = [geojson_toponimos.geometry[j] for j in indexs]
            new_elments = [(geojson_toponimos['toponimo_ocr'][row],
                            geojson_toponimos['confidence'][row],
                            geojson_toponimos['centroide_longitud'][row]) for row in indexs]

            new_elments.sort(key=lambda tup: tup[2],reverse=False)
            toponimo_ocr = [' '.join([i[0] for i in new_elments])]
            confidence = [max([i[1] for i in new_elments])]
            new_geom = [ cascaded_union( geom_list )  ]
            centroide_longitud = [new_geom[0].representative_point().coords[0][0] ]
            centroide_latitud = [new_geom[0].representative_point().coords[0][1] ]
            new_word = {'toponimo_ocr': toponimo_ocr,'confidence': confidence, 'centroide_longitud':centroide_longitud,
                        'centroide_latitud':centroide_latitud,'geometry':new_geom}

            aux_geojson = gpd.GeoDataFrame(new_word,columns=["toponimo_ocr","confidence","centroide_longitud",
                                                             "centroide_latitud","geometry"],crs=geojson_toponimos.crs)
            new_geojseon = new_geojseon.append(aux_geojson)

        geojson_toponimos = geojson_toponimos.drop(labels=indexs, axis=0).reset_index(drop=True)
        
    special_characters = ["<","(","[","{","^","-","=","$","!","|","]","}",")","?","*","+",".",">","¡"]
    new_geojseon.reset_index(drop=True,inplace=True)
    for count in range(len(new_geojseon)):
        string = new_geojseon["toponimo_ocr"][count]
        for character in special_characters:
            string = string.replace(character, "")
        
        split_string = string.split(" ")
        new_string = ""
        for token in split_string:
            if(token.strip().isdigit()):
                if( len( str(int( token.strip() )) )==1 ):
                    new_string = new_string +" "+ token
            else:
                new_string = new_string +" "+ token
        new_geojseon.at[count,"toponimo_ocr"] = new_string
    
    new_geojseon["toponimo_ocr"] = new_geojseon["toponimo_ocr"].apply(lambda x: "" if x.strip().isdigit() else x )
    new_geojseon = new_geojseon[new_geojseon["toponimo_ocr"]!=""].reset_index(drop=True)
    new_geojseon.to_file('text_detected.geojson',driver='GeoJSON')
    
    return new_geojseon

In [13]:
aerophotos = glob.glob("geotiffs/*.tif")
aerophotos

['geotiffs\\C-1974 F-238.tif',
 'geotiffs\\C-1974 F-240.tif',
 'geotiffs\\C-2070 F-250.tif',
 'geotiffs\\C-2070 F-252.tif',
 'geotiffs\\M-1390 F-42286.tif',
 'geotiffs\\M-1390 F-42290.tif']

## Suggested procedure

1. Delete all the temporary files (`empty_folders()`)
2. Create individual geometries from the path of the original image (`create_geometries(path)`)
3. Combine geometries into single geojson file (`combine_geometries()`)
4. Get coordinates of the corners of the original image (`get_image_corners(path)`)

### Testing for each of the 6 geotiff images

In [ ]:
# M-1390 F-42286
%%time
empty_folders()
create_geometries(aerophotos[0])
all_toponyms_img1 = combine_geometries(aerophotos[0])
all_toponyms_img1 = union_geometries(all_toponyms_img1)
get_image_corners(aerophotos[0])

In [53]:
%%time
all_toponyms_img1 = combine_geometries(aerophotos[0])
all_toponyms_img1 = union_geometries(all_toponyms_img1)

Wall time: 12.1 s
